# Library Creation

In [21]:
import keras
from keras.models import Sequential,Model
from tensorflow.keras.layers import Input
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import LeakyReLU
import tensorflow as tf
from tensorflow import keras
import keras.layers as layers
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [22]:
scale_percent = 100 # percent of original size
width = int(np.round(600 * scale_percent / 100))
height = int(np.round(300 * scale_percent / 100))
#dim = (width, height)

# Model Creation 

In [23]:
model = keras.Sequential()

# Layers of the Neural Network

In [24]:
model.add(layers.Conv2D(filters=96, kernel_size=(11, 11), 
                        strides=(4, 4), activation="relu", 
                        input_shape=(width, height, 3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(3, 3), strides= (2, 2)))
model.add(layers.Conv2D(filters=256, kernel_size=(5, 5), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
model.add(layers.Conv2D(filters=384, kernel_size=(3, 3), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(filters=384, kernel_size=(3, 3), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(filters=256, kernel_size=(3, 3), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(4096, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation="softmax"))

# Model compilation

In [46]:
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer=tf.optimizers.SGD(learning_rate=0.001), 
              metrics=['accuracy'])

# Model Summary

In [47]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 148, 73, 96)       34944     
                                                                 
 batch_normalization_5 (Batc  (None, 148, 73, 96)      384       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 73, 36, 96)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 73, 36, 256)       614656    
                                                                 
 batch_normalization_6 (Batc  (None, 73, 36, 256)      1024      
 hNormalization)                                                 
                                                      

# Dataset

In [48]:
train_dataset='C:/Code/Cross-view-attention/cross-view-attention/MINI-DDSM-Complete-JPEG-8'
valid_dataset='C:/Code/Cross-view-attention/cross-view-attention/MINI-DDSM-Complete-JPEG-8-valid'
test_dataset='C:/Code/Cross-view-attention/cross-view-attention/MINI-DDSM-Complete-JPEG-8-test'

# Training Generator

In [49]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=10
)

train_generator = train_datagen.flow_from_directory(
    train_dataset,
    target_size=(width,height), 
    interpolation="lanczos",
    color_mode='rgb', 
    batch_size=4, 
    class_mode='sparse', 
    shuffle=True)


Found 6588 images belonging to 3 classes.


# Validation Generator

In [50]:
# 'C:/Users/marcb/Downloads/miniddsm2/MINI-DDSM-Complete-JPEG-8-valid',
val_datagen = ImageDataGenerator(rescale=1. / 255) 	

valid_generator = val_datagen.flow_from_directory(
    valid_dataset,
    target_size=(width,height), 
    interpolation="lanczos",
    color_mode='rgb', 
    batch_size=4, 
    class_mode='sparse', 
    shuffle=True)
	

Found 620 images belonging to 3 classes.


# Model Training

In [31]:
from collections import Counter

In [51]:
model.fit(
        train_generator,
        steps_per_epoch=round( 6588   / 4 ),
        epochs=30,
        validation_data=valid_generator,
        validation_steps=round( 620 / 4 ), 
        workers=24)

Epoch 1/30
1647/1647 [==============================] - 1607s 974ms/step - loss: 1.8140 - accuracy: 0.5082 - val_loss: 0.9294 - val_accuracy: 0.5452
Epoch 2/30
1647/1647 [==============================] - 1555s 944ms/step - loss: 0.9210 - accuracy: 0.5469 - val_loss: 0.8720 - val_accuracy: 0.5306
Epoch 3/30
1647/1647 [==============================] - 1522s 923ms/step - loss: 0.8709 - accuracy: 0.5601 - val_loss: 0.7845 - val_accuracy: 0.6097
Epoch 4/30
1647/1647 [==============================] - 1499s 910ms/step - loss: 0.8445 - accuracy: 0.5735 - val_loss: 0.7753 - val_accuracy: 0.6048
Epoch 5/30
1373/1647 [========================>.....] - ETA: 4:06 - loss: 0.8211 - accuracy: 0.5859

KeyboardInterrupt: 

# Model Testing 

In [ ]:
test_datagen = ImageDataGenerator(
    rescale=1./255
)

test_generator = test_datagen.flow_from_directory(
    test_dataset,
    target_size=(width,height), 
    interpolation="lanczos",
    color_mode='rgb', 
    batch_size=4, 
    class_mode='sparse', 
    shuffle=True)

#y_pred=model.predict_on_batch(test_datagen)
test_evaluation = model.evaluate(test_generator,verbose = 0)

# Metrics

In [ ]:
for i in range(0,len(model.metrics_names)):
    print("%s%s: %.2f%%" % (" ",model.metrics_names[i], test_evaluation[i]*100))
